In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets

### loading iris dataset

In [63]:
iris=datasets.load_iris()
x = pd.DataFrame(iris.data)
x.columns = ["sl", "sw", 'pl', 'pw']

### converting the continious data into label data 
code copied from decision tree ML/extracthere/Assignment Decision Tree Sarthak/Decision_tree_assignment.ipynb

In [64]:
def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(x, old_feature_name):
    second = x[old_feature_name].mean()
    minimum = x[old_feature_name].min()
    first = (minimum + second)/2
    maximum = x[old_feature_name].max()
    third = (maximum + second)/2
    return x[old_feature_name].apply(label, args= (first, second, third))

x["s_l"]=toLabel(x,"sl")
x["s_w"]=toLabel(x,"sw")
x["p_l"]=toLabel(x,"pl")
x["p_w"]=toLabel(x,"pw")
x.drop(["sl","sw","pl","pw"],axis=1,inplace=True)

In [65]:
x.columns = ["sl", "sw", 'pl', 'pw']
x.head()

,sl,sw,pl,pw
0,b,c,a,a
1,a,b,a,a
2,a,c,a,a
3,a,c,a,a
4,a,c,a,a


In [66]:
arr = np.array(x)
print(len(arr[0]))
print(arr.shape)
(arr[:,0]=='a').sum()
col_names = ["sl", "sw", 'pl', 'pw']
col_names

4
(150, 4)


['sl', 'sw', 'pl', 'pw']

### FIT FUNCTION

   

In [67]:
def fit(xtrain,ytrain):
    dic ={}
    for curr_class in set(ytrain) :
        dic[curr_class] ={}
        dic["total_data"] = len(ytrain)
        
        # current_class_rows will have a boolean array with true when the particular row belongs to that classs
        current_class_rows = (ytrain==curr_class)
        # xtrain_for_curr_class has only those xtrain for a particular class
        xtrain_for_curr_class = xtrain[current_class_rows]
        ytrain_for_curr_class = ytrain[current_class_rows]
        # this stroes the length of that class ki total setosa kitna baaki kitne
        dic[curr_class]["total_count_of_each_class"] = len(ytrain_for_curr_class)
        
        # har class ke liye saare feature like sl,pl... etc pe traverse kro
        for j in range(xtrain.shape[1]):
            dic[curr_class][col_names[j]] = {}
            # all_values_a_feature_can_take will have all the distinct values a particular feature( like sepal length) 
            # for class 0 jaise sepal length can have a , b , c , d aise ..
            all_values_a_feature_can_take =  set(xtrain[:,j])
            
            # now for every value a , b, c,d for each class and every feature kitne usme se a y b h store krdo
            for distinct_value_of_feature in all_values_a_feature_can_take :    
                dic[curr_class][col_names[j]][distinct_value_of_feature] = (xtrain_for_curr_class[:,j]==distinct_value_of_feature).sum()   
          
    return dic
    
    
        
        

In [93]:
def getProb(dic,x,curr_class) :
    # formula P(y=a1)*p(xj=xj/y=a1)
    # class_prob stores p(y=a1) 
    # for class 0 it the total class 0 is 40 and class 0 + class 1 + class2 = 100
    # p(y=a1)  for class 0 will be 40/100
    # 40 is stored in total_count_of_each_class and 100 in total_data
    class_prob = np.log(dic[curr_class]['total_count_of_each_class']) - np.log(dic["total_data"])
    prob = class_prob
    # we have subtracted 1 because we have added one column of total_count_of_each_class
    num_of_features = len(dic[curr_class].keys()) - 1
    for j in range(num_of_features) :
        xj = x[j]
        # p(xj=xj/y=ya1)*p(y=a1)
        # now from above formula p(xj=xj/y=ya1) this equals to count where class =a1 and xj= xj which is stored in
        # count_with_curr_class_value_xj 
        ### LAPLACE CORRECTION DONE BY ADDING ! TO NUM AND
        ### COUNT OF DISTINCT VALUES EACH FEATURE CAN TAKE IN DENOMINARIOR
        ### WHICH IS GOT BY FETCHING ALL THE KEYS FOR CURR CLASS , CURR FEATURE
        count_with_curr_class_value_xj = dic[curr_class][col_names[j]][xj] + 1
        curr_class_count = dic[curr_class]['total_count_of_each_class'] + len(dic[curr_class][col_names[j]].keys())
        # p(xj=xj/y=ya1) = main_prob
        main_prob = np.log(count_with_curr_class_value_xj)-np.log(curr_class_count)
        prob = prob+main_prob
    
    return prob
    
        

In [94]:
def PredictForOne(dic,x):
    classes = dic.keys()
    # highest_prob stores like there is 33% chance that x belongs to class 0 and 40% chance that it belongs to class 1
    # and 80% chance that it belongs to class 2 so highest_prob will store 80
    highest_prob = -1000
    predicted_class = -123
    for curr_class in classes :
        if(curr_class =="total_data") :
            continue
        p_that_x_belongs_to_curr_class = getProb(dic,x,curr_class)
        if(p_that_x_belongs_to_curr_class>highest_prob) :
            highest_prob = p_that_x_belongs_to_curr_class
            predicted_class = curr_class
            
    return predicted_class
        

In [95]:
def predict(dic,xtest) :
    ypred=[]
    for x in xtest:
        x_class = PredictForOne(dic,x)
        ypred.append(x_class)  
    return ypred

In [96]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(np.array(x),iris.target)
dic = fit(xtrain,ytrain)
print(dic)

{0: {'total_count_of_each_class': 31, 'sl': {'c': 0, 'd': 0, 'b': 16, 'a': 15}, 'sw': {'c': 20, 'd': 8, 'b': 2, 'a': 1}, 'pl': {'c': 0, 'd': 0, 'b': 0, 'a': 31}, 'pw': {'c': 0, 'd': 0, 'b': 0, 'a': 31}}, 'total_data': 112, 1: {'total_count_of_each_class': 38, 'sl': {'c': 20, 'd': 2, 'b': 13, 'a': 3}, 'sw': {'c': 8, 'd': 0, 'b': 19, 'a': 11}, 'pl': {'c': 32, 'd': 0, 'b': 6, 'a': 0}, 'pw': {'c': 29, 'd': 0, 'b': 9, 'a': 0}}, 2: {'total_count_of_each_class': 43, 'sl': {'c': 25, 'd': 12, 'b': 5, 'a': 1}, 'sw': {'c': 13, 'd': 2, 'b': 23, 'a': 5}, 'pl': {'c': 18, 'd': 25, 'b': 0, 'a': 0}, 'pw': {'c': 14, 'd': 29, 'b': 0, 'a': 0}}}


In [97]:


class_prob = np.log(dic[curr_class]['total_count_of_each_class']) - np.log(dic["total_data"])

In [98]:
ypred = predict(dic,np.array(xtest))
ypred

[0,
 1,
 1,
 2,
 1,
 0,
 1,
 1,
 2,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1]

In [99]:
from sklearn.metrics import classification_report , confusion_matrix

In [100]:
print(classification_report(ytest,ypred))
print(confusion_matrix(ytest,ypred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       0.86      1.00      0.92        12
           2       1.00      0.71      0.83         7

    accuracy                           0.95        38
   macro avg       0.95      0.90      0.92        38
weighted avg       0.95      0.95      0.95        38

[[19  0  0]
 [ 0 12  0]
 [ 0  2  5]]
